In [3]:
import urllib.parse
import urllib.request
import pandas as pd
import pickle
import re
import numpy as np

In [4]:
def NotBruteAtAll(temp):
    res = []
    for t in temp:
        result = [] #ответ в шаблоне ФИО
        t = t.lower()
        words = t.split(" ")
        
        resultStrict = StrictCheck(words)
        order, check = СheckStrictMatrix(resultStrict)
        if check:
            print(result)
            for i in range(3):
                result.append(words[order[i]])   
        # + установить, что если частота меньше N проверить реплейсом (т.к. только редкие, э-экономия)
        else:
            resultReplWords, resultReplValues = ReplaceCheck(words)
            resultGramma = GrammaCheck(words) 
            replaced = np.array(resultReplValues) + np.array(resultGramma)+ np.array(statistics)
            strict = np.array(resultStrict) + np.array(resultGramma)+ np.array(statistics)
            print(replaced)
            print(strict)
            print("---")
            strictOrder, stictWords, strictValues = GetMaxWordsValues(strict,[words]*3)
            replacedOrder, replacedWords, replacedValues = GetMaxWordsValues(replaced,resultReplWords)
            
            for i in range(3):
                if (strictValues[strictOrder[i]]>replacedValues[replacedOrder[i]]):
                    result.append(stictWords[strictOrder[i]])
                    order[i] = strictOrder[i]
                else:
                    result.append(replacedWords[replacedOrder[i]])
                    order[i] = replacedOrder[i]
        SetStatistics(order)
        result = result[0].title() + " " + result[1].title() + " " + result[2].title()
        #result = result.title()
        res.append(result)
    return res

In [6]:
def GetOrder(matrix):
    order = [0,0,0]
    for j in range(3):
        for i in range(3):
            if (matrix[i][j]!=0):
                order[j] = i
                break
    return order

In [7]:
def СheckMatrix(matrix): #Если у нас в одной строке и столбце только по одному элементу, то возвращает True
    for i in range(3):
        k2 = 0
        for j in range(3):
            if (matrix[i][j] != 0): k2+=1
        if (k2 != 1):
            return False
    return True

In [8]:
def СheckStrictMatrix(matrix): #проверяет
    for i in range(3):
        for j in range(3):
            if(matrix[i][j]>0): matrix[i][j]=1
    matrix = recursiveChange(matrix, None)
    return GetOrder(matrix), СheckMatrix(matrix)

In [9]:
def recursiveChange(matrix, matrixOld): #изменяет значения, за подробностями в лс
    if(matrix==matrixOld): return matrix
    matrixOld = matrix
    counters = [0,0,0]
    for i in range(3):
        counters[i] = matrix[i].count(1)
        
    if counters.count(1)==3: return matrix
    if counters.count(1)==2 and counters.count(0)==1:
        counters2 = [False,False,False]
        for j in range(3):
            flag = True
            for i in range(3):
                if (matrix[i][j] != 0):
                    flag = False
                    break
            counters2[j] = flag
        i = counters.index(0)
        j = counters2.index(True)
        matrix[i][j] = 1
        return recursiveChange(matrix, matrixOld)
    
    if counters.count(1)==1:
        k = counters.index(1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
        return recursiveChange(matrix, matrixOld)
    if (counters.count(1)==2 
                                and matrix[counters.index(1)].index(1)!=
            matrix[counters.index(1, counters.index(1)+1)].index(1)):
        k = counters.index(1, counters.index(1)+1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
        
        k = counters.index(1)
        j = matrix[k].index(1)
        for i in range(3):
            if (i != k):
                matrix[i][j] = 0
                
        return recursiveChange(matrix, matrixOld)
    return matrix

In [10]:
def GetMaxWordsValues(matrixValues, matrixWords):
    order = [None,None,None]
    words = [None,None,None]
    values = [0,0,0]
    for j in range(3):
        maxI = 0
        for i in range(3):
            if (matrixValues[i][j]>matrixValues[maxI][j]):
                maxI = i
        if (maxI not in order):
            order[j] = maxI
            words[j] = matrixWords[maxI][j]
            values[j] = matrixValues[maxI][j]
        #else: return False #исправить
        else:
            order[j] = 0
            words[j] = ""
            values[j] = 0
    return order, words, values

In [11]:
def StrictCheck(words): #строгая проверка
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]
    for i in range(3):
        w = words[i].strip()
        if (w in all_surnames): 
            result[i][0] = all_surnames[w]    
        if (w in all_names):
            result[i][1] = all_names[w]          
        if (w in all_patronymics):
            result[i][2] = all_patronymics[w] 
    return result

In [12]:
def ReplaceCheck(words): #проверка с перестановками
    #          F I O
    resultWords = [[0,0,0],
                  [0,0,0],
                  [0,0,0]]
    resultValues = [[0,0,0],
                  [0,0,0],
                  [0,0,0]]
    for i in range(3):
        w = words[i].strip()
        resultWords[i][0], resultValues[i][0] = forReplaceCheck(w, all_surnames)
        resultWords[i][1], resultValues[i][1] = forReplaceCheck(w, all_names) 
        resultWords[i][2], resultValues[i][2] = forReplaceCheck(w, all_patronymics)
    return resultWords, resultValues

In [13]:
def forReplaceCheck(w, list): #отдельный метод, составляет списки слов с 1 и 2 расстоянием
    res = [{"": 0},{"": 0}]
    for l in list:
        dist = shortDistance(w, l)
        if (dist != False):
            res[int(dist)-1][l] = list[l]*10**(-1*dist)

    key1, value1 = max(res[0].items(), key=lambda x:x[1])
    key2, value2 = max(res[1].items(), key=lambda x:x[1])
    
    if (value1>value2):
        return key1, value1
    else: return key2, value2

In [14]:
def shortDistance(word, dictWord): #упрощенное расстояние (поиск только с опечаткой 1 и 2)
    #word - слово которое проверяем
    #dictWord - слово из списка, с которым сравниваем
    if word == dictWord: return 0
    elif len(word) == 0: return len(dictWord)
    elif len(dictWord) == 0: return len(word)
    elif len(dictWord) > len(word)+2 and len(dictWord) < len(word)+2:
        return False
    dist = levenshtein(word, dictWord)
    #dist = damerau(word, dictWord)
    if (dist == 1 or dist==2):
        return dist
    else: return False

In [15]:
def levenshtein(s, t): #расстояние Левенштейна
    if s == t: return 0
    elif len(s) == 0: return len(t)
    elif len(t) == 0: return len(s)
    v0 = [None] * (len(t) + 1)
    v1 = [None] * (len(t) + 1)   
    for i in range(len(v0)):
        v0[i] = i
    for i in range(len(s)):
        v1[0] = i + 1
        for j in range(len(t)):
            cost = 0 if s[i] == t[j] else 1
            v1[j + 1] = min(v1[j] + 1, v0[j + 1] + 1, v0[j] + cost)
        for j in range(len(v0)):
            v0[j] = v1[j]
    return v1[len(t)]

In [16]:
def damerau(s, t): #расстояние Дамерау-Левенштейна (с перестановкой)
    deleteCost = 1
    insertCost = 1
    replaceCost = 1
    transposeCost = 1
    s = " "+s
    t = " "+t
    M = len(s)
    N = len(t)
    d = [list(range(N))]
    for i in range(1,M):
        d.append([])
        for j in range(N):
            d[i].append(0)
        d[i][0] = i
        
    for i in range(1,M):
        for j  in range(1,N):          
            # Стоимость замены
            if (s[i] == t[j]):
                d[i][j] = d[i-1][j-1]
            else:
                d[i][j] = d[i-1][j-1] + replaceCost                   
            d[i][j] = min(
                             d[i][j],                               # замена
                             d[i-1][j] + deleteCost,                # удаление
                             d[i][j-1] + insertCost                 # вставка               
                         )
            if(i > 1 and j > 1 and s[i] == t[j-1] and s[i-1] == t[j]):
                d[i][j] = min(
                                  d[i][j],
                                  d[i-2][j-2] + transposeCost         # транспозиция
                             )
    return d[M-1][N-1]

In [17]:
def GrammaCheck(words): #проверка по грамматике
    #          F I O
    result = [[0,0,0],
              [0,0,0],
              [0,0,0]]

    for i in range(3):
        w = words[i].strip()
        result[i][0] = checkSurnames(w)
        result[i][2] = checkPatronymic(w)

    k = 0
    for i in range(3):
        flag = True
        for j in range(3):
            if (result[i][j] != 0): 
                flag = False 
                break
        if(flag):
            result[i][1] = 1
            k+=1
    if(k==0):
        for i in range(3): result[i][1] = 0.33
    elif(k>1): 
        for i in range(3): result[i][1] = result[i][1]/k
    return result

In [18]:
def checkSurnames(s):#раньше возвращало тру фолс, теперь 1 или 0
    pattern = '\w*(ов|ова|ев|ёв|ева|ёва|ив|ин|ина|ын|их|ых|ский|цкий|ая|ко|дзе|онок|ян)$'
    if (re.match(pattern,s)): return 1
    else: return 0

In [19]:
def checkPatronymic(s):
    pattern = '\w*(ович|евич|ич|овна|евна|ична|инична)$'
    if (re.match(pattern,s)):
        return 1
    else: return 0

In [20]:
#               F     I    O
statistics = [[0.50,0.45,0.05],
              [0.10,0.50,0.40],
              [0.40,0.05,0.55]]
# FIO: 0.50, IOF: 0.40, IFO: 0.15, ElseOptions: 0.05
#пока что совсем от балды, простите
def SetStatistics(order):
    for i in range(3):
        for j in range(3):
            if (j == order[i]): 
                statistics[i][j] += 0.002
            else: statistics[i][j] -= 0.001
#Обновляет статистику с учетом результата выполнения алгоритма            

In [21]:
path_to_surnames = "D:/all_surnames.pickle"
path_to_names = "D:/all_names.pickle"
path_to_patronymics = "D:/all_patronymics.pickle"
with open(path_to_surnames,"rb") as f:
    all_surnames=pickle.load(f)
with open(path_to_names,"rb") as f:
    all_names=pickle.load(f)
with open(path_to_patronymics,"rb") as f:
    all_patronymics=pickle.load(f)   

In [24]:
# with open("D:/test_changed.pickle","rb") as f:
#     tmp=pickle.load(f)
#tmp = ["Бучнева Анна Всеволодовна", "Иванов Иван Иванович"]

inputText = ["Петров Кирилл Анатольевич",
             "Александр Иванович Сидоров",
             "Бугазявкин Петре Петрович",
             "Синюевич Анатольевич Иван",
             "Иванович Иванов Иван",
             "Сидорок Анатоилй Петрович",
             "Кириллович Иван Соколов"]
testing_res=NotBruteAtAll(inputText)
for fio in testing_res:
    print(fio)

[]
[]


[[  1.50300000e+00   4.44000000e-01   5.30000000e-02]
 [  1.00001030e+04   3.39015030e+04   3.94000000e-01]
 [  4.80040000e+03   2.67000000e-01   3.39155300e+03]]
[[ 1.503  0.444  0.053]
 [ 0.103  1.503  0.394]
 [ 1.4    0.047  2.553]]
---


[[  7.00505000e+02   4.43000000e-01   1.90520000e+01]
 [  2.01050000e+01   5.02000000e-01   1.00139300e+03]
 [  7.00003990e+04   6.07046000e+02   2.00055500e+03]]
[[ 0.505  0.443  1.052]
 [ 0.105  0.502  1.393]
 [ 1.399  2.046  0.555]]
---
[]


[[  9.50050300e+03   7.09440000e+01   7.00530000e+01]
 [  1.04600000e+00   6.93000000e+02   3.94000000e-01]
 [  4.80040300e+03   2.64000000e-01   3.39155300e+03]]
[[ 0.503  0.944  0.053]
 [ 0.106  1.     0.394]
 [ 1.403  0.044  2.553]]
---
[]
Петров Кирилл Анатольевич
Сидоров Александр Иванович
Бугазявкин Перу Петровна
Иван Сан Сан
Иванов Иван Иванович
Сидоров Анатолий Петровна
Соколов Иван Кириллович


In [50]:
def NotBruteAtAllOld(tmp): #болванка для метода, причем старая
    res = []
    for t in temp:
        result = [] #ответ в шаблоне ФИО
        
        resultStrict = StrictCheck(t)
        if СheckMatrix(resultStrict):
            order = GetOrder(result)
            for i in range(3):
                result.append(words[order[i]])   
        # + установить, что если частота меньше N проверить реплейсом (т.к. только редкие, э-экономия)
        else:
            resultRepl = ReplaceCheck(t)
            resultReplValues = resultRepl.values()
            
            resultGramma = GrammaCheck(t) 
            #Сделать грамма чек если только все остальное пусто 
            # ИЛИ мы сравнили строгий результат и гибкий, и там все непонятно
            #Потому что иначе, если уже найдено, то зачем проверять: если ошибка в написании, она может быть
            resultStatistics = statistics
        #разобраться с Ё и Е
        #result = np.array(result)
        #подсчитываем все с матрицами
        #принимаем решение, записываем в рез
        #вызываем метод SetStatistics и обновляем статистику порядка
        
        res.append(result)
    return res

In [51]:
def NotBruteEnough(tmp):
    res = []

    for t in tmp:
        name = ""
        surname = ""
        patronymic = ""
        t = t.lower()
        words = t.split(" ")

        for w in words:
            w.strip()
            if (name == ""):
                if (w in all_names):
                    name = w

            if (patronymic == ""):
                if (w in all_patronymics):
                    patronymic = w

            if (name != w and patronymic != w and checkSurnames(w) and surname == ""):
                surname = w

        if (name in words): words.remove(name)
        if (patronymic in words): words.remove(patronymic)
        if (surname in words): words.remove(surname)

        if (name == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for n in all_names:
                    dist = levenshtein(w, n)
                    if dist < minDist:
                        minDist = dist
                        name = n.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (patronymic == "") & (len(words) > 0):
            foundWord = ""
            for w in words:
                minDist = len(w)
                for p in all_patronymics:
                    dist = levenshtein(w, p)
                    if dist < minDist:
                        minDist = dist
                        patronymic = p.title() #+ " (" + str(round((1 - minDist / len(w)) * 100)) + "% " + w + ")"
                        foundWord = w
            words.remove(foundWord)

        if (surname == "")& (len(words) > 0):
            surname = words[0]

        name = name.title()
        surname = surname.title()
        patronymic = patronymic.title()
        res.append(surname + " " + name + " " + patronymic)

    return res

In [52]:
#genders Пока не используем
genderTuple = ('male', 'female', 'unknown')

def checkSurnamesGender(s):
    patternMale = '\w*(ов|ев|ий|ын|ин)$'
    patternFem ='\w*(ова|ева|ая|ина|ына)$'
    patternUnknown = '\w*(их|ых|ко|ук|юк|ун|ний|ный|чай|ий|а|ич|ов|ук|ик|ски|ка|ски|цки|дзки)$'
    
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    elif (re.match(patternUnknown,s)):
        return genderTuple[2]
    else: return 'No'

def changeGenderSurname(s):
    patternMaleA = '\w*(ов|ев|ин|ын)$'
    patternFemA ='\w*(ова|ева|ина|ына)$'
    patternMaleB = '\w*(ий)$'
    patternFemB = '\w*(ая)$'
    
    if(re.match(patternMaleA, s)):
        return s + 'a'
    elif (re.match(patternMaleB, s)):       
        s += ' '
        return s.replace('ий ', 'ая')
    elif (re.match(patternFemA, s)):
        return s[0: len(s)-1]
    elif (re.match(patternFemB, s)): 
        s += ' '
        return s.replace('ая ', 'ий')
    else: return s

def checkPatronymicGender(s):
    patternMale = '\w*(ович|евич|ич)$'
    patternFem ='\w*(овна|евна|ична|инична)$'
   
    if (re.match(patternMale,s)):
        return genderTuple[0]
    elif (re.match(patternFem,s)):
        return genderTuple[1]
    else: return 'No'    

def changeGenderPatronymic(s):
    patternMaleA = '\w*(ович|евич)$'
    patternFemA ='\w*(овна|евна)$'
    patternMaleB = '\w*(ич)$'
    patternFemB = '\w*(ична|инична)$'

    if(re.match(patternMaleA, s)):
        s += ' '
        return s.replace('ич ', 'на')
    if(re.match(patternFemA, s)):
        s += ' '
        return s.replace('на ', 'ич')     
    if(re.match(patternMaleB, s)):
        return s + 'на'   
    if(re.match(patternFemB, s)):
        s += ' '
        s = s.replace('инична ', 'ич')
        s = s.replace('ична ', 'ич')
        return s    
    else: return s

In [35]:
def WorkingWithPickle():
    path_to_names = "D:/NamesFromGufo.pickle"
    path_to_patronymics = "D:/PatronymicsFromGufo.pickle"
    path_to_new_names = "D:/names_extended.pickle"
    path_to_new_surnames = "D:/surnames_extended.pickle"
    #all_names = []
    patronymics = []
    names_extended = {}
    all_surnames = {}
    
    # with open(path_to_names,"rb") as f:
    #     all_names=pickle.load(f)
    with open(path_to_patronymics,"rb") as f:
        patronymics=pickle.load(f)    
    with open(path_to_new_names,"rb") as f:
        names_extended=pickle.load(f)
    with open(path_to_new_surnames,"rb") as f:
        all_surnames=pickle.load(f)
    all_names = {}
    all_patronymics = {}
    for i in names_extended:
        all_names[i] = names_extended[i][1]
    for j in patronymics:
        for i in all_names:
            if (j.find(i)!= -1):
                all_patronymics[j] = all_names[i]
    with open("all_patronymics.pickle","wb") as f:
        pickle.dump(all_patronymics,f)
    with open("all_surnames.pickle","wb") as f:
        pickle.dump(all_surnames,f)
    with open("all_names.pickle","wb") as f:
        pickle.dump(all_names,f)